<a href="https://colab.research.google.com/github/THESHAMIK/EDA---Explolatory-Data-Analysis-for-ML/blob/main/Deep_NN/KHDS_Deep_Keras%2BScikitLearn_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Regression Tutorial for KHDS with the **Keras Deep Learning** Library in Python with blend of Scikit-learn. scikit-learn excels at evaluating models so this blend is used.

Keras is a deep learning library that wraps the efficient numerical libraries Theano and TensorFlow. 

Here we will learn ::

How to use **scikit-learn with Keras** to evaluate models using cross-validation.
How to perform **data preparation** in order to improve skill with Keras models.
How to **tune the network **topology of models with Keras.

We can **create Keras models** and **evaluate them with scikit-learn** by using handy **wrapper** objects provided by the Keras library. 

This is desirable, because **scikit-learn excels at evaluating models** and will allow us to use powerful data preparation and model evaluation schemes with very few lines of code.

REFERENCE ::: Keras_TF_Ski_1.ipynb
#Regression Problem solved with the Keras Deep Learning + SKLearn Library in Python
#https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/
#Here basic model used, then input data SCALED, then TUNNED version for both DEEPER & WIDER layers used to get better result.

In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_csv("khds_final_1.csv")

In [ ]:
#U can see last column Product Sulpur(Y) is an object & not Float
#data['Product Sulphur']=data['Product Sulphur'].astype(float64) ##==> # This is NOT working
df['Product Sulphur'] = pd.to_numeric(df['Product Sulphur'],errors = 'coerce') ##==> This one works as pd.to_numeric works for convert a list, a series, an array, or a tuple to a numeric datatype
#type(data['Product Sulphur']) ##This shows that Product Sulphur data type is a series hence astype did not work
df.dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 34 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   LABfeed Sulphur                            711 non-null    float64
 1   T1 : FRESH FEED FLOW TO KHDS KGPerHour     711 non-null    float64
 2   T1 : Reactor inlet temperature DEGC        711 non-null    float64
 3   T1 : Hydrogen make-up flow NM3perHour      711 non-null    float64
 4   T1 : Recycle hydrogen flow NM3perHour      711 non-null    float64
 5   T1 : REACTOR OUTLET Temp DEGC              711 non-null    float64
 6   T1 : REACTOR FEED INLET Pressure KGperCM2  711 non-null    float64
 7   T1 : Cold separator pressure KGperCM2      711 non-null    float64
 8   T1 : KERO FEED DENSITY KGperCM2            711 non-null    float64
 9   T2 : FRESH FEED FLOW TO KHDS KGPerHour     711 non-null    float64
 10  T2 : Reactor inlet tempera

The missing data in Last_Name is represented as None and the missing data in Age is represented as NaN, Not a Number. This is because pandas handles the missing values in numeric as NaN and other objects as None. Don’t worry, pandas deals with both of them as missing values

In [ ]:
filtered_df = df[df['Product Sulphur'].notnull()]

Forced to use filtered_df as df.dropna() was NOT working below. My aim is to remove NULL values in Product Sulphur col as below baseline mean was coming as 0. This happens when there is any null value in any column.
So Keras dont remove them U have to.

sklearn.model_selection import cross_val_score dont handle NULL values well. One has to do this manually.

In [ ]:
df.dropna(axis=0)

,LABfeed Sulphur,T1 : FRESH FEED FLOW TO KHDS KGPerHour,T1 : Reactor inlet temperature DEGC,T1 : Hydrogen make-up flow NM3perHour,T1 : Recycle hydrogen flow NM3perHour,T1 : REACTOR OUTLET Temp DEGC,T1 : REACTOR FEED INLET Pressure KGperCM2,T1 : Cold separator pressure KGperCM2,T1 : KERO FEED DENSITY KGperCM2,T2 : FRESH FEED FLOW TO KHDS KGPerHour,...,T3 : KERO FEED DENSITY KGperCM2,T4 : FRESH FEED FLOW TO KHDS KGPerHour,T4 : Reactor inlet temperature DEGC,T4 : Hydrogen make-up flow NM3perHour,T4 : Recycle hydrogen flow NM3perHour,T4 : REACTOR OUTLET Temp DEGC,T4 : REACTOR FEED INLET Pressure KGperCM2,T4 : Cold separator pressure KGperCM2,T4 : KERO FEED DENSITY KGperCM2,Product Sulphur
0,1000.0,72482.43948,288.485914,685.146132,11114.58673,288.637635,21.927602,18.775248,0.000000,72337.37151,...,0.000000,72353.77432,287.978641,614.163076,11226.93106,288.712160,21.805388,18.666758,0.000000,2.0
1,1400.0,75588.69117,286.406314,571.248477,11259.14433,286.921575,21.911739,18.702087,0.000000,75746.39266,...,0.000000,75918.19844,287.134693,464.921691,11066.97566,286.615329,21.934672,18.747208,0.000000,0.5
2,1300.0,75675.29792,284.035734,564.716667,10668.45313,284.140627,21.900282,18.734029,0.000000,75628.86875,...,0.000000,76002.57969,284.800492,580.681543,10701.71289,284.840621,21.848295,18.704427,0.000000,3.0
3,1500.0,46269.02109,276.210773,371.162782,14497.57829,276.808471,21.747162,19.528809,0.000000,46001.59297,...,0.000000,46176.64115,275.822623,565.165267,14783.88027,276.540142,21.547435,19.357554,0.000000,4.0
4,1400.0,45394.43555,276.126803,296.835800,14606.11231,276.776767,21.560785,19.492783,0.000000,45231.54465,...,0.000000,45265.78914,276.090728,409.074912,14675.20297,277.132889,21.597432,19.496689,0.000000,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,1100.0,60139.03926,269.888446,511.399785,12541.86891,271.360266,21.023166,18.624649,694.466242,59989.55111,...,695.525236,60417.56009,270.061212,581.807254,12470.92394,272.810638,20.995547,18.596115,694.614817,19.0
707,1200.0,60362.18822,269.971474,558.111740,12542.77303,272.554566,21.018436,18.601252,697.014856,60543.49408,...,696.144514,60372.60352,270.042269,501.625794,12590.90794,272.603820,21.103575,18.656947,696.279553,20.0
708,1200.0,60425.38188,270.062989,528.413451,12517.83553,272.306049,21.014648,18.605937,693.293790,60522.48813,...,693.736376,60353.13302,270.083814,536.398630,12618.29635,272.570469,21.040742,18.611755,693.730737,25.0
709,1300.0,64871.35271,270.130087,561.401982,12287.34497,272.229078,21.142253,18.609416,697.181100,66726.60260,...,697.110177,69193.52375,269.897449,575.264706,12380.30654,272.226493,21.223976,18.602946,698.950006,28.0


In [ ]:
#df.info()
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 709 entries, 0 to 710
Data columns (total 34 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   LABfeed Sulphur                            709 non-null    float64
 1   T1 : FRESH FEED FLOW TO KHDS KGPerHour     709 non-null    float64
 2   T1 : Reactor inlet temperature DEGC        709 non-null    float64
 3   T1 : Hydrogen make-up flow NM3perHour      709 non-null    float64
 4   T1 : Recycle hydrogen flow NM3perHour      709 non-null    float64
 5   T1 : REACTOR OUTLET Temp DEGC              709 non-null    float64
 6   T1 : REACTOR FEED INLET Pressure KGperCM2  709 non-null    float64
 7   T1 : Cold separator pressure KGperCM2      709 non-null    float64
 8   T1 : KERO FEED DENSITY KGperCM2            709 non-null    float64
 9   T2 : FRESH FEED FLOW TO KHDS KGPerHour     709 non-null    float64
 10  T2 : Reactor inlet tempera

In [ ]:
target_col = "Product Sulphur"
X = filtered_df.loc[:, filtered_df.columns != target_col]
Y = filtered_df.loc[:, target_col]

We can **create Keras models** and **evaluate them with scikit-learn** by using handy **wrapper** objects provided by the Keras library. 

This is desirable, because **scikit-learn excels at evaluating models** and will allow us to use powerful data preparation and model evaluation schemes with very few lines of code.

In [ ]:
X.shape

(709, 33)

In [ ]:
Y.shape

(709,)

The network uses good practices such as the **rectifier activation** function for the hidden layer. No activation function is used for the output layer because it is a regression problem and we are interested in predicting numerical values directly without transform.

The efficient ADAM optimization algorithm is used and a mean squared error loss function is optimized. This will be the same metric that we will use to evaluate the performance of the model. It is a desirable metric because by taking the square root gives us an error value we can directly understand in the context of the problem

In [ ]:
# define base model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(33, input_dim=33, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [ ]:
# evaluate model
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, Y, cv=kfold, error_score="raise")
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Baseline: -41.13 (37.20) MSE


Pl note above, Baseline mean was coming as nan as there were 2 NULL values in Product Sulphur col. I have to remove that using filtered_df above & bring count from 711 to 709.

The **input attributes all vary in their scales** because they measure different quantities.

It is almost always good practice to prepare your data before modeling it using a neural network model.

Continuing on from the above baseline model, we can re-evaluate the same model using a standardized version of the input dataset.

In [ ]:
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  after removing the cwd from sys.path.


Standardized: -255.29 (716.33) MSE


So above, standarlizing the data using Scalar proved very bad. The MSE failed from 41 to 255. This is in line with the output from PYCARET also where no normalization or even transformation of Y did any help. Basic setup gave us best output. Pl check analysis of KHDS for pycaret.

Lets try diff method then. Lets **Tune The Neural Network Topology** using 2 options:::
 1.  Evaluate a **Deeper** Network Topology
 2. Evaluate a **Wider** Network Topology

In [ ]:
# define the model
def larger_model():
	# create model
	model = Sequential()
	model.add(Dense(33, input_dim=33, kernel_initializer='normal', activation='relu'))
	model.add(Dense(6, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [ ]:
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  after removing the cwd from sys.path.


Larger: -496.18 (1446.44) MSE


In [ ]:
# evaluate model
estimator = KerasRegressor(build_fn=larger_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, Y, cv=kfold, error_score="raise")
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Baseline: -20.47 (27.60) MSE


Huge IMPROVEMENT of MSE from 41 to 20!! Amazing just increasing one Deep N/W.
Now lets try the WIDER option.

In [ ]:
# define wider model
def wider_model():
	# create model
	model = Sequential()
	model.add(Dense(66, input_dim=33, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [ ]:
# evaluate model
estimator = KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, Y, cv=kfold, error_score="raise")
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Baseline: -48.21 (45.02) MSE


WIder n/w did not perform so well. Lets stick to Deeper N.W

In [ ]:
# define the model
def larger_model_1():
  # create model
  model = Sequential()
  model.add(Dense(33, input_dim=33, kernel_initializer='normal', activation='relu'))
  model.add(Dense(20, kernel_initializer='normal', activation='relu'))
  model.add(Dense(9, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal'))
  # Compile model
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

In [ ]:
# evaluate model - HERE also changed batch_size = 20 from 5
estimator = KerasRegressor(build_fn=larger_model_1, epochs=100, batch_size=20, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, Y, cv=kfold, error_score="raise")
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Baseline: -20.79 (28.82) MSE


No change seen in MSE after adding one more layer in deeper side.